In [ ]:
%load_ext autoreload
%autoreload 2

import asyncio
import math
import time

from ipywidgets import (
    HBox,
    VBox,
    Button,
    IntSlider,
    Dropdown,
    Checkbox,
    ToggleButton,
    Label
)
from ipyevents import Event

from park.simulation import Simulation
from park.world import World
from park.render import Camera, Renderer
from park.stats import Curve

from park.internal.math import Vector2D
from park.internal.physics import Physics
from park.logic.queue import VisitorQueue

In [ ]:
world_width = 512
world_height = 512

cam_width = 512
cam_height = 512

cell_size = 64

world = World(
    width=world_width,
    height=world_height,
    cell_size=cell_size,
    space_type=World.SpaceType.CONTINUOUS,
)
sim = Simulation(
    world=world,
    rng=None,
)

ride_configs = [
    (
        "Roller Coaster", # name
        Vector2D(cell_size, cell_size), # position
        40, # capacity
        900, # duration in steps
        VisitorQueue( # entrance
            world,
            head=Vector2D(cell_size, cell_size * 1.25),
            tail=Vector2D(cell_size, cell_size * 3),
            spacing=max(cell_size // 16, 4.0)
        ),
        VisitorQueue( # exit
            world,
            head=Vector2D(cell_size * 3, cell_size),
            tail=Vector2D(cell_size * 1.25, cell_size),
            spacing=max(cell_size // 16, 4.0),
        )
    ),
    (
        "Ferris Wheel",
        Vector2D(world_width - cell_size, cell_size),
        30,
        600,
        VisitorQueue(
            world,
            head=Vector2D(world_width - cell_size, cell_size * 1.25),
            tail=Vector2D(world_width - cell_size, cell_size * 3),
            spacing=max(cell_size // 16, 4.0)
        ),
        VisitorQueue(
            world,
            head=Vector2D(world_width - cell_size * 3, cell_size),
            tail=Vector2D(world_width - cell_size * 1.25, cell_size),
            spacing=max(cell_size // 16, 4.0),
        )
    ),
    (
        "Carousel",
        Vector2D(world_width // 2, world_height // 2 - cell_size // 2),
        15,
        500,
        VisitorQueue(
            world,
            head=Vector2D(world_width // 2, world_height // 2 - cell_size * 0.25),
            tail=Vector2D(world_width // 2, world_height // 2 + cell_size * 1.5),
            spacing=max(cell_size // 16, 4.0)
        ),
        VisitorQueue(
            world,
            head=Vector2D(world_width // 2, world_height // 2 - cell_size * 2.5),
            tail=Vector2D(world_width // 2, world_height // 2 - cell_size * 0.75),
            spacing=max(cell_size // 16, 4.0)
        )
    ),
    (
        "Bumper Cars",
        Vector2D(cell_size, world_height - cell_size * 2),
        25,
        550,
        VisitorQueue(
            world,
            head=Vector2D(cell_size * 1.25, world_height - cell_size * 2),
            tail=Vector2D(cell_size * 3, world_height - cell_size * 2),
            spacing=max(cell_size // 16, 4.0)
        ),
        VisitorQueue(
            world,
            head=Vector2D(cell_size, world_height - cell_size * 4),
            tail=Vector2D(cell_size, world_height - cell_size * 2.25),
            spacing=max(cell_size // 16, 4.0),
        )
    ),
    (
        "Drop Tower",
        Vector2D(world_width - cell_size, world_height - cell_size * 2),
        35,
        800,
        VisitorQueue(
            world,
            head=Vector2D(world_width - cell_size * 1.25, world_height - cell_size * 2),
            tail=Vector2D(world_width - cell_size * 3, world_height - cell_size * 2),
            spacing=max(cell_size // 16, 4.0)
        ),
        VisitorQueue(
            world,
            head=Vector2D(world_width - cell_size, world_height - cell_size * 4),
            tail=Vector2D(world_width - cell_size, world_height - cell_size * 2.25),
            spacing=max(cell_size // 16, 4.0),
        )
    )
]
for name, pos, capacity, duration, entrance_queue, exit_queue in ride_configs:
    sim.spawn_ride(name, pos, capacity, duration, entrance_queue, exit_queue)

for _ in range(10):
    sim.spawn_robot()

cam = Camera(
    width=cam_width,
    height=cam_height,
    world_width=world_width,
    world_height=world_height,
    min_zoom=1.0,
    max_zoom=1.0
)

plot_size = Vector2D(120, 80)
ren = Renderer(
    sim,
    cam,
    pixel_scale=cell_size,
    tile_grouping=0,
)
ren.add_curve(Curve(
    "Money",
    max_samples=1000,
    line_color="#4cc181",
    value_format="{:.0f}",
    auto_scale=True
))
ren.add_curve(Curve(
    "Satisfaction",
    max_samples=1000,
    line_color="#ffb347",
    value_format="{:.2f}",
    auto_scale=False,
    floor=0.0,
    ceil=1.0,
))
ren.add_curve(Curve(
    "In Queues",
    max_samples=1000,
    line_color="#b732c4",
    value_format="{:.0f}",
    auto_scale=True
))
ren.add_curve(Curve(
    "On Rides",
    max_samples=1000,
    line_color="#32aaff",
    value_format="{:.0f}",
    auto_scale=True
))

In [ ]:
start_btn = Button(description="Start", button_style="success")
stop_btn = Button(description="Stop",  button_style="warning")
info = Label("Mouse drag to pan, mouse wheel to zoom")

speed_selector = Dropdown(
    options=[
        ("0.5x", 0.5),
        ("1x", 1.0),
        ("2x", 2.0),
        ("3x", 3.0),
        ("5x", 5.0)
    ],
    value=1.0,
    description="Speed Factor",
    indent=False
)
def on_speed_change(change):
    global _sim_base_fps, _sim_speed, _sim_fps, _sim_dt
    _sim_speed = float(change["new"])
    _sim_fps = _sim_base_fps * _sim_speed
    _sim_dt = 1.0 / _sim_fps
speed_selector.observe(on_speed_change, names="value")

ren.set_stats_visible(True)
show_stats = ToggleButton(
    value=True,
    indent=False,
    button_style="info",
    description="Hide Stats",
    tooltip="Toggles stats visibility"
)
def on_show_stats_change(change):
    ren.set_stats_visible(change["new"])
    show_stats.description = "Hide Stats" if change["new"] else "Show Stats"
show_stats.observe(on_show_stats_change, names="value")

ren.set_colliders_visible(False)
show_colliders = ToggleButton(
    value=False,
    indent=False,
    button_style="info",
    description="Show Colliders",
    tooltip="Toggles colliders visibility"
)
def on_show_colliders_change(change):
    ren.set_colliders_visible(change["new"])
    show_colliders.description = "Hide Colliders" if change["new"] else "Show Colliders"
show_colliders.observe(on_show_colliders_change, names="value")

ui = VBox([
    ren.widget,
    HBox([
        speed_selector,
        show_stats,
        show_colliders,
    ]),
    # HBox([
        # start_btn,
        # stop_btn,
        # speed,
    # ]),
    # info,
])

In [ ]:
# --- pan/zoom state ---
_dragging = False
_last_xy = (0.0, 0.0)
_acc_dx = 0.0
_acc_dy = 0.0

# --- simulation config ---
_sim_base_fps = 60.0
_sim_speed = 1.0
_sim_fps = _sim_base_fps * _sim_speed
_sim_dt = 1.0 / _sim_fps
_sim_task = None

# --- physics config ---
_physics_fps = 60.0
_physics_dt = 1.0 / _physics_fps
_physics_task = None

# --- rendering config ---
_target_fps = 60.0
_min_dt = 1.0 / _target_fps
_last_frame_t = 0.0
_render_task = None

async def _sim_loop():
    """Steps the simulation at a fixed rate."""
    while True:
        sim.step()
        await asyncio.sleep(_sim_dt)

async def _physics_loop():
    """Steps physics at a fixed rate."""
    while True:
        Physics.step(_physics_dt * _sim_speed)
        await asyncio.sleep(_physics_dt)

async def _render_loop():
    """Render at ~target_fps; also consume accumulated pan deltas."""
    global _acc_dx, _acc_dy, _last_frame_t, _dragging
    _last_frame_t = 0.0
    while True:
        now = time.monotonic()
        if now - _last_frame_t >= _min_dt:
            if _dragging:
                dx, dy = _acc_dx, _acc_dy
                _acc_dx = _acc_dy = 0.0
                if dx or dy:
                    # screen px -> world units; drag moves the view
                    cam.pan(-(dx) / cam.zoom, -(dy) / cam.zoom)

            ren.update_curve("Satisfaction", sim.get_avg_satisfaction())
            ren.update_curve("In Queues", sim.get_members_in_queues())
            ren.update_curve("On Rides", sim.get_members_on_rides())

            ren.update_draw()
            _last_frame_t = now
        await asyncio.sleep(_min_dt)

# Attach pointer + wheel events to the MultiCanvas widget
_ev = Event(
    source=ren.mcanvas,
    watched_events=[
        "mousedown",
        "mouseup",
        "mousemove",
        "wheel",
    ],
    prevent_default_action=True,
    stop_propagation=True,
)

def _on_dom_event(e: dict):
    global _dragging, _last_xy, _acc_dx, _acc_dy, _render_task

    t = e.get("type", "")
    x = float(e.get("offsetX", 0.0))
    y = float(e.get("offsetY", 0.0))

    if t == "mousedown":
        _dragging = (e.get("buttons", 0) & 1) != 0  # left button
        _last_xy = Vector2D(x, y)
        # start a loop if not running
        if (_render_task is None) or _render_task.done():
            _render_task = asyncio.create_task(_render_loop())
        return

    if t == "mouseup":
        _dragging = False
        return

    if t == "mousemove" and _dragging:
        _acc_dx += (x - _last_xy.x)
        _acc_dy += (y - _last_xy.y)
        _last_xy = Vector2D(x, y)
        return

    if t == "wheel":
        dy = float(e.get("deltaY", 0.0))
        ren.zoom_at(Vector2D(x, y), dy)
        ren.update_draw()

# _ev.on_dom_event(_on_dom_event)

if (_sim_task is None) or _sim_task.done():
    _sim_task = asyncio.create_task(_sim_loop())  # start sim loop

if (_physics_task is None) or _physics_task.done():
    _physics_task = asyncio.create_task(_physics_loop())  # start physics loop

if (_render_task is None) or _render_task.done():
    _render_task = asyncio.create_task(_render_loop())  # start render loop

In [ ]:
ui